# Mise en place de spark & kafka

### Mise en place du Producer

In [72]:
from pyspark.sql import SparkSession
from pyspark.sql.functions import json_tuple

from kafka import KafkaConsumer, KafkaProducer
import json
import os

def create_spark_session():
    spark = SparkSession.builder \
        .appName("KafkaSpark") \
        .getOrCreate()
    return spark

def produce_messages(message, topic):
    producer = KafkaProducer(
        bootstrap_servers='kafka1:29092, kafka2:29092, kafka3:29092,',
        value_serializer=lambda v: json.dumps(v).encode('utf-8')
    )

    producer.send('add_to_es', value=message)
    
    producer.flush()
    producer.close()
    

### Mise en place du Consumer

In [73]:
def consume_messages(spark, topic):
    consumer = KafkaConsumer(
        topic,
        bootstrap_servers='localhost:29092',
        value_deserializer=lambda m: json.loads(m.decode('utf-8'))
    )

    for message in consumer:
        process_message(spark, message.value)

def process_message(spark, message):
    print("Processing message: ", message)
    # Here you can add logic to insert the data into MongoDB
    # For example, df.write.format("mongo").mode("append").save("mongodb://mongo:[MONGOPORT]/your_db.your_collection")

### Test du consumer & producer

In [74]:
spark = create_spark_session()

Test producer

In [ ]:
from kafka.errors import NoBrokersAvailable

try:
	produce_messages({"name": "John", "age": 30}, 'add_to_es')
except NoBrokersAvailable as e:
	print("Error: No Kafka brokers available. Please ensure the Kafka broker is running and the address is correct.")
	print(e)

Test Consumer

In [ ]:
try:
    consume_messages(spark, 'add_to_es')
except NoBrokersAvailable as e:
    print("Error: No Kafka brokers available. Please ensure the Kafka broker is running and the address is correct.")
    print("Detailed error message:", e)
except Exception as e:
    print("An unexpected error occurred:", e)